### Importacion de librerias

In [478]:
### Importacion de archivos
import pandas as pd
import numpy as np

### Carga de archivos

In [479]:
clientes = pd.read_csv('current_reports/clientes.csv')
df_clientes = pd.DataFrame(clientes)

### Manipulación de reporte de clientes

In [480]:
df_clientes = df_clientes.drop(df_clientes[df_clientes["Sucursal"] == "Sucursal Coyoacan"].index)
df_clientes = df_clientes[['index', 'pos_client', 'reg_dtt', 'client_type','name', 'Nombre', 'Apellido', 'email', 'is_active', 'phone', 'region', 'Sucursal',
                          'steel_discount', 'lamin_discount','buy_back_enabled','csg_enabled']]
df_clientes.rename(columns={'index': "BN-Cliente",'Apellido': 'Apellidos', 'pos_client': "POS-Cliente", 'reg_dtt': 'F/h registro',
                            'client_type': 'Tipo', 'email': 'Correo', 'is_active': 'Web', 'region': 'Region',
                            'lamin_discount': 'Desc Laminado', 'steel_discount': 'Desc Acero', 'buy_back_enabled': 'Re-compra',
                            'csg_enabled': 'Consigna', 'phone': 'Teléfono'}, inplace=True)


# Rellenando datos faltantes
df_clientes['Nombre'] = df_clientes['Nombre'].fillna('<NULL>')
df_clientes['Apellidos'] = df_clientes['Apellidos'].fillna('<NULL>')
df_clientes['name'] = df_clientes['name'].fillna('<NULL>')
df_clientes['POS-Cliente'] = df_clientes['POS-Cliente'].fillna('<NULL>')
df_clientes['Correo'] = df_clientes['Correo'].fillna('<NULL>')
df_clientes['Nombre'] = df_clientes['Nombre'].map(lambda x: "" if ( x == "<NULL>" ) else x)
df_clientes['Apellidos'] = df_clientes['Apellidos'].map(lambda x: "" if ( x == "<NULL>" ) else x)
df_clientes['name'] = df_clientes['name'].map(lambda x: "" if ( x == "<NULL>" ) else x)
df_clientes['Nombre Completo'] = df_clientes['Nombre'] + " " + df_clientes['Apellidos']
df_clientes['Nombre Completo'] = np.where(df_clientes['Nombre Completo'] == ' ', df_clientes['name'], df_clientes['Nombre Completo']) 
df_clientes['Nombre Completo'] = np.where(df_clientes['Nombre Completo'] == '', None, df_clientes['Nombre Completo']) 

# Definiendo tipos de dato
df_clientes['POS-Cliente'] = df_clientes['POS-Cliente'].astype('string')
df_clientes['Tipo'] = df_clientes['Tipo'].astype('string')
df_clientes['Nombre'] = df_clientes['Nombre'].astype('string')
df_clientes['Apellidos'] = df_clientes['Apellidos'].astype('string')
df_clientes['Correo'] = df_clientes['Correo'].astype('string')
df_clientes['Web'] = df_clientes['Web'].astype('bool')
df_clientes['Teléfono'] = df_clientes['Teléfono'].astype('string')
df_clientes['Region'] = df_clientes['Region'].astype('string')
df_clientes['Sucursal'] = df_clientes['Sucursal'].astype('string')
df_clientes['Re-compra'] = df_clientes['Re-compra'].astype('bool')
df_clientes['Consigna'] = df_clientes['Consigna'].astype('bool')


### Unificación de nombres de cliente

In [481]:
# Formateando nombres
df_clientes['Apellidos'] = df_clientes['Apellidos'].str.upper()
df_clientes['Nombre'] = df_clientes['Nombre'].str.upper()
df_clientes['Nombre Completo'] = df_clientes['Nombre Completo'].str.upper()
df_clientes['name'] = df_clientes['name'].str.upper()

# Archivo para unificacion de nombres de clientes
with pd.ExcelWriter('new_reports/unificacionNombres.xlsx') as writer:
    df_clientes[["BN-Cliente", "Nombre Completo", "Nombre", "Apellidos"]].to_excel(writer, index=False)

### Desgloce de clientes sin numero telefonico

In [482]:
df_clientes['Teléfono'] = df_clientes['Teléfono'].fillna('<NULL>')
df_clientes['Teléfono'] = df_clientes['Teléfono'].map(lambda x: x if len(x) == 10 else '<NULL>')
df_clientes_not_phone = df_clientes[df_clientes['Teléfono'] == "<NULL>"]
df_clientes_not_phone = df_clientes_not_phone[['BN-Cliente', 'POS-Cliente', 'F/h registro', 'Nombre Completo', 'Correo', 
                                               'Teléfono', 'Region', 'Sucursal']]
df_clientes_not_phone['BN-Cliente'] = np.where(df_clientes_not_phone['POS-Cliente'] == '<NULL>', "Si", "No")
df_clientes_not_phone['POS-Cliente'] = np.where(df_clientes_not_phone['POS-Cliente'] == '<NULL>', "No", "Si")
df_clientes_not_phone['Correo'] = np.where(df_clientes_not_phone['Correo'] == '<NULL>', 'No', 'Si')
df_clientes_not_phone['Nombre Completo'] = np.where(df_clientes_not_phone['Nombre Completo'] == '', "No", "Si")
count_correo = df_clientes_not_phone['Correo'].value_counts()
count_pos = df_clientes_not_phone['POS-Cliente'].value_counts()
count_bn = df_clientes_not_phone['BN-Cliente'].value_counts()
count_name = df_clientes_not_phone['Nombre Completo'].value_counts()
count_region = df_clientes_not_phone['Region'].value_counts()
# count_store = df_clientes_not_phone['Sucursal'].value_counts()
df_resumen = pd.DataFrame([count_name, count_correo, count_bn, count_pos, count_region]).fillna(0)

# Archivo de clientes sin numero o numero invalido
with pd.ExcelWriter('new_reports/telefonos_invalidos.xlsx') as writer:
    df_clientes.to_excel(writer, index=False)

df_resumen

,Si,No,CDMX,GDL,<NULL>
Nombre Completo,1711.0,0.0,0.0,0.0,0.0
Correo,79.0,1632.0,0.0,0.0,0.0
BN-Cliente,69.0,1642.0,0.0,0.0,0.0
POS-Cliente,1642.0,69.0,0.0,0.0,0.0
Region,0.0,0.0,1459.0,235.0,17.0
